In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Датасет содержит данные о поездках в такси.
Провести эксплоративный анализ данных (проверки на дубликаты, пустые значение, описательные статистики, распределения, корреляции между показателями и т.д.)
С помощью регрессионного анализа восстановить формулу тарификации поездок на основании предложенных данных: сначала с помощью дисперсионного анализа определить влияние компонентов цены на итоговую цену, затем построить регрессию и оценить ее точность.
Подготовить визуализацию и интерпретацию результатов.

In [3]:
df = pd.read_csv('/content/drive/MyDrive/hakaton/data.csv')

In [5]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-02-13 04:14:08,2016-02-13 04:28:47,1,4.90,-73.985497,40.747047,1,N,-73.964272,40.802654,1,16.5,0.5,0.5,2.00,0.0,0.3,19.80
1,2,2016-02-12 00:16:03,2016-02-12 00:26:55,1,2.50,-73.944473,40.794769,1,N,-73.917809,40.815128,2,10.5,0.5,0.5,0.00,0.0,0.3,11.80
2,2,2016-02-13 17:55:46,2016-02-13 18:00:17,3,0.26,-74.014168,40.712559,1,N,-74.012688,40.713905,2,4.5,0.0,0.5,0.00,0.0,0.3,5.30
3,2,2016-02-11 22:58:36,2016-02-11 23:24:56,1,5.65,-73.985107,40.759430,1,N,-73.914146,40.765759,1,21.5,0.5,0.5,4.56,0.0,0.3,27.36
4,2,2016-02-10 22:00:24,2016-02-10 22:03:03,1,0.64,-73.971870,40.791855,1,N,-73.966835,40.799110,2,4.0,0.5,0.5,0.00,0.0,0.3,5.30


In [9]:
df.isna().sum()

,0
VendorID,0
tpep_pickup_datetime,0
tpep_dropoff_datetime,0
passenger_count,0
trip_distance,0
pickup_longitude,0
pickup_latitude,0
RatecodeID,0
store_and_fwd_flag,0
dropoff_longitude,0


Нет NaN'ов, и на том спасибо. Проверим на нули и постараемся восстановить места, где нулей быть не должно. Это passenger_count, trip_distance, широта и долгота, total_amount, налоги, fare_amount

In [8]:
print((df == 0).sum())

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count               7
trip_distance               863
pickup_longitude           2519
pickup_latitude            2519
RatecodeID                    0
store_and_fwd_flag            0
dropoff_longitude          2369
dropoff_latitude           2369
payment_type                  0
fare_amount                  46
extra                     75725
mta_tax                     584
tip_amount                52028
tolls_amount             142898
improvement_surcharge        27
total_amount                 12
dtype: int64


In [10]:
dist = np.sqrt((df['dropoff_latitude'] - df['pickup_latitude'])**2 + (df['dropoff_longitude'] - df['pickup_longitude'])**2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
165,2,2016-02-11 13:36:27,2016-02-11 13:36:42,1,0.0,-73.936913,40.764511,1,N,-73.936974,40.764542,3,-2.5,0.0,-0.5,0.00,0.0,-0.3,-3.30
487,2,2016-02-10 16:20:33,2016-02-10 16:21:57,1,0.0,0.000000,0.000000,1,N,0.000000,0.000000,2,1.1,0.0,0.5,0.00,0.0,0.3,1.90
537,1,2016-02-09 06:54:18,2016-02-09 06:54:18,1,0.0,-73.997604,40.756683,1,N,0.000000,0.000000,2,2.5,0.0,0.5,0.00,0.0,0.3,3.30
541,1,2016-02-08 12:44:53,2016-02-08 12:45:23,1,0.0,-73.859932,40.683556,3,N,-73.859940,40.683544,4,20.0,0.0,0.0,0.00,0.0,0.3,20.30
911,2,2016-02-08 08:44:51,2016-02-08 08:44:54,1,0.0,0.000000,0.000000,5,N,-73.870857,40.774090,1,50.0,0.0,0.5,10.16,0.0,0.3,60.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149554,2,2016-02-11 17:28:03,2016-02-11 17:28:18,1,0.0,-73.788620,40.641579,2,N,-73.788620,40.641575,1,52.0,4.5,0.5,11.46,0.0,0.3,68.76
149772,1,2016-02-11 07:56:42,2016-02-11 07:56:42,1,0.0,-73.897713,40.768143,1,N,0.000000,0.000000,2,6.0,0.0,0.5,0.00,0.0,0.3,6.80
149812,2,2016-02-09 22:44:25,2016-02-09 22:44:29,1,0.0,0.000000,0.000000,5,N,-73.960831,40.774872,1,15.0,0.0,0.0,3.06,0.0,0.3,18.36
149965,1,2016-02-10 18:47:10,2016-02-10 18:47:52,1,0.0,-73.987373,40.744576,1,N,-73.988274,40.743401,2,2.5,1.0,0.5,0.00,0.0,0.3,4.30
